#  Postrgres SQL Finnish


Source of data is Mozill common Voice Data Set
I am using google colab as it has most of libraries intalled e. g ffmpeg

---



## .Mount drive

In [ ]:
#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Update to tables library-pandas need that to avoid write errors

In [ ]:
!pip install --upgrade tables

     |████████████████████████████████| 4.3 MB 7.6 MB/s 
  Attempting uninstall: tables
    Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


## . Loading libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## . Loading data set from Mozilla

### 1.Extraction of files to further  processing

In [ ]:
!tar  xvf /content/drive/MyDrive/pipeline/fi.tar.gz

cv-corpus-6.1-2020-12-11/fi/
cv-corpus-6.1-2020-12-11/fi/clips/
cv-corpus-6.1-2020-12-11/fi/dev.tsv
cv-corpus-6.1-2020-12-11/fi/invalidated.tsv
cv-corpus-6.1-2020-12-11/fi/other.tsv
cv-corpus-6.1-2020-12-11/fi/reported.tsv
cv-corpus-6.1-2020-12-11/fi/test.tsv
cv-corpus-6.1-2020-12-11/fi/train.tsv
cv-corpus-6.1-2020-12-11/fi/validated.tsv
cv-corpus-6.1-2020-12-11/fi/clips/common_voice_fi_22986631.mp3
cv-corpus-6.1-2020-12-11/fi/clips/common_voice_fi_22986632.mp3
cv-corpus-6.1-2020-12-11/fi/clips/common_voice_fi_22986633.mp3
cv-corpus-6.1-2020-12-11/fi/clips/common_voice_fi_22986634.mp3
cv-corpus-6.1-2020-12-11/fi/clips/common_voice_fi_22986635.mp3
cv-corpus-6.1-2020-12-11/fi/clips/common_voice_fi_22987456.mp3
cv-corpus-6.1-2020-12-11/fi/clips/common_voice_fi_22987458.mp3
cv-corpus-6.1-2020-12-11/fi/clips/common_voice_fi_22987459.mp3
cv-corpus-6.1-2020-12-11/fi/clips/common_voice_fi_22987462.mp3
cv-corpus-6.1-2020-12-11/fi/clips/common_voice_fi_22987463.mp3
cv-corpus-6.1-2020-12-11/fi/cl

In [ ]:
!cp /content/cv-corpus-6.1-2020-12-11/fi/validated.tsv /content/drive/MyDrive/pipeline
!cp /content/cv-corpus-6.1-2020-12-11/fi/invalidated.tsv  /content/drive/MyDrive/pipeline

In [ ]:
df_val= pd.read_csv('/content/drive/MyDrive/pipeline/validated.tsv',  sep='\t')
df_inval=pd.read_csv('/content/drive/MyDrive/pipeline/invalidated.tsv', sep='\t')


In [ ]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1305 entries, 0 to 1304
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   client_id   1305 non-null   object 
 1   path        1305 non-null   object 
 2   sentence    1305 non-null   object 
 3   up_votes    1305 non-null   int64  
 4   down_votes  1305 non-null   int64  
 5   age         1049 non-null   object 
 6   gender      1049 non-null   object 
 7   accent      0 non-null      float64
 8   locale      1305 non-null   object 
 9   segment     0 non-null      float64
dtypes: float64(2), int64(2), object(6)
memory usage: 102.1+ KB


In [ ]:
df_inval.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   client_id   59 non-null     object 
 1   path        59 non-null     object 
 2   sentence    59 non-null     object 
 3   up_votes    59 non-null     int64  
 4   down_votes  59 non-null     int64  
 5   age         30 non-null     object 
 6   gender      30 non-null     object 
 7   accent      0 non-null      float64
 8   locale      59 non-null     object 
 9   segment     0 non-null      float64
dtypes: float64(2), int64(2), object(6)
memory usage: 4.7+ KB


## .Checking for missing values starndard checking...

In [ ]:
missing_values= df_val.isnull().sum().sort_values(ascending= False)/len(df_val)
print(len(missing_values[missing_values>0]), 'are with missing values' )
print(missing_values[missing_values>0])

4 are with missing values
segment    1.000000
accent     1.000000
gender     0.196169
age        0.196169
dtype: float64


In [ ]:
df_val.sample(5)

,client_id,path,sentence,up_votes,down_votes,age,gender,accent,locale,segment
1288,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,common_voice_fi_23804627.mp3,Minä rakastan sinua,2,0,thirties,male,NaN,fi,NaN
779,7839f10ac19c4c5448e198442f4cd7fbfacea0cc2f9d63...,common_voice_fi_23347929.mp3,"Meillä on oikeus ja velvollisuus rakentaa, kun...",2,0,twenties,male,NaN,fi,NaN
313,fb0d052dcb96d413197d7eb19d6b6c7d63d64f2f4d8d82...,common_voice_fi_22996757.mp3,Juutaksen hyvä tuuli katkesi.,2,0,NaN,NaN,NaN,fi,NaN
762,7839f10ac19c4c5448e198442f4cd7fbfacea0cc2f9d63...,common_voice_fi_23347897.mp3,Bulgarialaiset rannat ovat valkoisia.,2,0,twenties,male,NaN,fi,NaN
1230,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,common_voice_fi_23800747.mp3,Sitten huudetaan karjapihalla Malakiasta.,2,0,thirties,male,NaN,fi,NaN


## Removing unnecessary columns

In [ ]:
list(df_val.columns)

['client_id',
 'path',
 'sentence',
 'up_votes',
 'down_votes',
 'age',
 'gender',
 'accent',
 'locale',
 'segment']

In [ ]:

del df_val['up_votes']
del df_val['down_votes']
del df_val['segment']

del df_inval['up_votes']
del df_inval['down_votes']
del df_inval['segment']


In [ ]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1305 entries, 0 to 1304
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   client_id  1305 non-null   object 
 1   path       1305 non-null   object 
 2   sentence   1305 non-null   object 
 3   age        1049 non-null   object 
 4   gender     1049 non-null   object 
 5   accent     0 non-null      float64
 6   locale     1305 non-null   object 
dtypes: float64(1), object(6)
memory usage: 71.5+ KB


In [ ]:
df_inval.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   client_id  59 non-null     object 
 1   path       59 non-null     object 
 2   sentence   59 non-null     object 
 3   age        30 non-null     object 
 4   gender     30 non-null     object 
 5   accent     0 non-null      float64
 6   locale     59 non-null     object 
dtypes: float64(1), object(6)
memory usage: 3.4+ KB


## Column validation status

In [ ]:
df_val['validation_status']='valid'
df_inval['validation_status']='invalid'

In [ ]:
df_val.sample(3)

,client_id,path,sentence,age,gender,accent,locale,validation_status
1213,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,common_voice_fi_23750412.mp3,Aistin ilmapiirin olevan tukala.,thirties,male,NaN,fi,valid
749,7839f10ac19c4c5448e198442f4cd7fbfacea0cc2f9d63...,common_voice_fi_23178084.mp3,"Minä, hupsu, ostin pilkallani.",twenties,male,NaN,fi,valid
1157,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,common_voice_fi_23391576.mp3,"On selvää, että sisällä olemme turvassa, mutta...",thirties,male,NaN,fi,valid


In [ ]:
df_inval.sample(3)

,client_id,path,sentence,age,gender,accent,locale,validation_status
14,fb0d052dcb96d413197d7eb19d6b6c7d63d64f2f4d8d82...,common_voice_fi_22996395.mp3,Finian katsoi minua vähättelevästi:,NaN,NaN,NaN,fi,invalid
17,fb0d052dcb96d413197d7eb19d6b6c7d63d64f2f4d8d82...,common_voice_fi_22996455.mp3,Ei rinnan röyhistelyä.,NaN,NaN,NaN,fi,invalid
26,fb0d052dcb96d413197d7eb19d6b6c7d63d64f2f4d8d82...,common_voice_fi_22996892.mp3,Pyörittelin hassun muotoisia hedelmiä käsissän...,NaN,NaN,NaN,fi,invalid


## Removing teens

to avoid problems with filtering letsfill na assign tyopes etc age has 19% missing (checked at the begining)


In [ ]:
{x:' ' for x in list(df_val.columns)}

{'accent': ' ',
 'age': ' ',
 'client_id': ' ',
 'gender': ' ',
 'locale': ' ',
 'path': ' ',
 'sentence': ' ',
 'validation_status': ' '}

In [ ]:
fillna_dict= {'accent': ' ',
 'age': '0',
 'client_id': ' ',
 'gender': ' ',
 'locale': ' ',
 'path': ' ',
 'sentence': ' ',
 'validation_status': ' '}

In [ ]:
df_val= df_val.fillna(fillna_dict)
df_inval= df_inval.fillna(fillna_dict)

In [ ]:
df_val.sample(3)

,client_id,path,sentence,age,gender,accent,locale,validation_status
295,fb0d052dcb96d413197d7eb19d6b6c7d63d64f2f4d8d82...,common_voice_fi_22996595.mp3,Ilmastonmuutos vaikuttaa yhteiskuntaan monin t...,0,,,fi,valid
617,03350aa4d6bc224e6f519b1ae7b225b66707194de28978...,common_voice_fi_23803408.mp3,Ovi kävi ja hän vavahti.,fourties,male,,fi,valid
738,7839f10ac19c4c5448e198442f4cd7fbfacea0cc2f9d63...,common_voice_fi_23178073.mp3,"Olinhan minä hänet ennenkin nähnyt, mutta nimi...",twenties,male,,fi,valid


In [ ]:
df_inval.sample(3)

,client_id,path,sentence,age,gender,accent,locale,validation_status
0,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,common_voice_fi_22987459.mp3,Ja tupaa voisi kyllä lämmittää.,thirties,male,,fi,invalid
26,fb0d052dcb96d413197d7eb19d6b6c7d63d64f2f4d8d82...,common_voice_fi_22996892.mp3,Pyörittelin hassun muotoisia hedelmiä käsissän...,0,,,fi,invalid
15,fb0d052dcb96d413197d7eb19d6b6c7d63d64f2f4d8d82...,common_voice_fi_22996397.mp3,Maunon Sanelma hihittää.,0,,,fi,invalid


In [ ]:
missing_values= df_val.isnull().sum().sort_values(ascending= False)/len(df_val)
print(len(missing_values[missing_values>0]), 'are with missing values' )
print(missing_values[missing_values>0])

0 are with missing values
Series([], dtype: float64)


looks ok with na

In [ ]:
{x:'str' for x in list(df_val.columns)}

{'accent': 'str',
 'age': 'str',
 'client_id': 'str',
 'gender': 'str',
 'locale': 'str',
 'path': 'str',
 'sentence': 'str',
 'validation_status': 'str'}

In [ ]:
fill_astype_dict={'accent': 'str',
 'age': 'str',
 'client_id': 'str',
 'gender': 'str',
 'locale': 'str',
 'path': 'str',
 'sentence': 'str',
 'validation_status': 'str'}

In [ ]:
df_val= df_val.astype(fill_astype_dict)
df_inval= df_inval.astype(fill_astype_dict)


In [ ]:
df_val.age.value_counts()

thirties    536
0           256
fourties    242
twenties    207
teens        54
fifties      10
Name: age, dtype: int64

so just remove teens

In [ ]:
df_val= df_val[~df_val['age'].str.contains('teens')]
df_inval= df_inval[~df_inval['age'].str.contains('teens')]

In [ ]:
df_val.age.value_counts()

thirties    536
0           256
fourties    242
twenties    207
fifties      10
Name: age, dtype: int64

In [ ]:
df_inval.age.value_counts()

0           29
thirties    19
twenties     6
fourties     2
Name: age, dtype: int64

## Convertting audio files mp3 to wav into sanitasied audio folder

1. lets do some function of converting mp3 to wav
2. make a list of mp3 direstory as a input for this function
do converting


In [ ]:
!cp /content/cv-corpus-6.1-2020-12-11/fi/clips/* /content/drive/MyDrive/pipeline/temp

cp: cannot stat '/content/cv-corpus-6.1-2020-12-11/fi/clips/*': No such file or directory


In [ ]:
import os

In [ ]:
def recode_mp3_from_folder(path):
  to_recode = [path+s  for s in os.listdir(path) if s.endswith('.mp3')]

  #creating list of tuples source and result name
  to_recode_out=[]
  for i in to_recode:
    to_recode_out.append((i, i[:-4]+'recode.wav'))
    
  #
  for  j in to_recode_out:
    print(j[0], ' --> processing ',j[1] )
    os.system(f"ffmpeg -i {j[0]} -ar 32000 -ac 1 {j[1]}")
   

In [ ]:
# ffmpeg -i input.mp3 -ar 8000 -ac 1 output.wav

# starting after test




In [ ]:
# ffmpeg -i input.mp3 output.wav

In [ ]:
# to_recode = [path+s  for s in os.listdir(path) if s.endswith('.mp3')]

In [ ]:
# to_recode[0]

'/content/drive/MyDrive/pipeline/temp/common_voice_fi_22986631.mp3'

In [ ]:
# to_recode_out=[]
# for i in to_recode:
#   to_recode_out.append((i,'/content/drive/MyDrive/pipeline/sanitized_audio/'+i[-28:]+'recode.wav'))

In [ ]:
to_recode_out

[('/content/drive/MyDrive/pipeline/temp/common_voice_fi_22986631.mp3',
  '/content/drive/MyDrive/pipeline/sanitized_audio/common_voice_fi_22986631.mp3recode.wav'),
 ('/content/drive/MyDrive/pipeline/temp/common_voice_fi_22986632.mp3',
  '/content/drive/MyDrive/pipeline/sanitized_audio/common_voice_fi_22986632.mp3recode.wav'),
 ('/content/drive/MyDrive/pipeline/temp/common_voice_fi_22986633.mp3',
  '/content/drive/MyDrive/pipeline/sanitized_audio/common_voice_fi_22986633.mp3recode.wav'),
 ('/content/drive/MyDrive/pipeline/temp/common_voice_fi_22986634.mp3',
  '/content/drive/MyDrive/pipeline/sanitized_audio/common_voice_fi_22986634.mp3recode.wav'),
 ('/content/drive/MyDrive/pipeline/temp/common_voice_fi_22986635.mp3',
  '/content/drive/MyDrive/pipeline/sanitized_audio/common_voice_fi_22986635.mp3recode.wav'),
 ('/content/drive/MyDrive/pipeline/temp/common_voice_fi_22987456.mp3',
  '/content/drive/MyDrive/pipeline/sanitized_audio/common_voice_fi_22987456.mp3recode.wav'),
 ('/content/driv

In [ ]:
pathout = '/content/drive/MyDrive/pipeline/sanitized_audio/'
path ='/content/drive/MyDrive/pipeline/temp/'

ffmpeg -i input.mp3 -ar 8000 -ac 1 output.wav

In [ ]:
from tqdm import tqdm
import json

In [ ]:
def recode_mp3_from_folder(path, pathout):
  to_recode = [path+s  for s in os.listdir(path) if s.endswith('.mp3')]

  #creating list of tuples source and result name
  to_recode_out=[]
  for i in to_recode:
    to_recode_out.append((i,pathout+i[-28:]+'recode.wav'))
    
  #
  for  j in to_recode_out: #tqdm is slowing down so it will be without tqdm(to_recode_out)
    print(j[0], ' --> processing ',j[1] )
    os.system(f"ffmpeg -i {j[0]} {j[1]}")

In [ ]:
recode_mp3_from_folder(path, pathout)

  0%|          | 0/1513 [00:00<?, ?it/s]

/content/drive/MyDrive/pipeline/temp/common_voice_fi_23997033.mp3  --> processing  /content/drive/MyDrive/pipeline/sanitized_audio/common_voice_fi_23997033.mp3recode.wav


  0%|          | 1/1513 [00:02<51:28,  2.04s/it]

/content/drive/MyDrive/pipeline/temp/common_voice_fi_23997034.mp3  --> processing  /content/drive/MyDrive/pipeline/sanitized_audio/common_voice_fi_23997034.mp3recode.wav


  0%|          | 2/1513 [00:02<31:10,  1.24s/it]

...

100%|█████████▉| 1511/1513 [18:08<00:01,  1.40it/s]

/content/drive/MyDrive/pipeline/temp/common_voice_fi_23997031.mp3  --> processing  /content/drive/MyDrive/pipeline/sanitized_audio/common_voice_fi_23997031.mp3recode.wav


100%|█████████▉| 1512/1513 [18:09<00:00,  1.41it/s]

/content/drive/MyDrive/pipeline/temp/common_voice_fi_23997032.mp3  --> processing  /content/drive/MyDrive/pipeline/sanitized_audio/common_voice_fi_23997032.mp3recode.wav


100%|██████████| 1513/1513 [18:09<00:00,  1.39it/s]


to test in temp directory resul;t delete etc

## Information about file in json format including duration which is returned as int

In [ ]:
import json

In [ ]:
def mp4_info(input_video):
  input_video= '/content/drive/MyDrive/pipeline/temp/'+input_video
  json_file= input_video[:-4]+'_params.json'
  os.system(f"ffprobe -v quiet -print_format json -show_format -show_streams {input_video} > {json_file}")
  json1 = json.loads(open(json_file).read())
  value = json1['streams'][0]['duration']
  duration=int(float(value))
  return duration

In [ ]:
# a= mp4_info('common_voice_fi_24001125.mp3')
# a

7

### lets do some apply and this function which calulates  time duration and update column

In [ ]:
df_val['audio_duration']= df_val.path.apply(lambda row: mp4_info(row))
df_inval['audio_duration']= df_inval.path.apply(lambda row: mp4_info(row))

In [ ]:
df_val.sample(3)

,client_id,path,sentence,age,gender,accent,locale,validation_status,audio_duration
284,d73b6988ab0d28d09702ed9ba80088137e7f9dec002169...,common_voice_fi_23212824.mp3,Hän vetäytyi takaisin oven luokse.,fourties,male,,fi,valid,3
309,fb0d052dcb96d413197d7eb19d6b6c7d63d64f2f4d8d82...,common_voice_fi_22996725.mp3,"Hän tietenkin arvasi, mitä minä ajattelin, vää...",0,,,fi,valid,7
608,03350aa4d6bc224e6f519b1ae7b225b66707194de28978...,common_voice_fi_23803399.mp3,Meillä oli mökki alhaalla harjun rinteellä.,fourties,male,,fi,valid,4


In [ ]:
df_inval.sample(3)

,client_id,path,sentence,age,gender,accent,locale,validation_status,audio_duration
36,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,common_voice_fi_23091533.mp3,Hän on kyllä hämäräperäinen henkilö!,thirties,male,,fi,invalid,2
24,fb0d052dcb96d413197d7eb19d6b6c7d63d64f2f4d8d82...,common_voice_fi_22996561.mp3,"Ihan itkeskeli, kotia kaipasi.",0,,,fi,invalid,3
1,fb0d052dcb96d413197d7eb19d6b6c7d63d64f2f4d8d82...,common_voice_fi_22996236.mp3,Samoin tunteet ja ajatukset.,0,,,fi,invalid,3


In [ ]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1251 entries, 0 to 1304
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   client_id          1251 non-null   object
 1   path               1251 non-null   object
 2   sentence           1251 non-null   object
 3   age                1251 non-null   object
 4   gender             1251 non-null   object
 5   accent             1251 non-null   object
 6   locale             1251 non-null   object
 7   validation_status  1251 non-null   object
 8   audio_duration     1251 non-null   int64 
dtypes: int64(1), object(8)
memory usage: 97.7+ KB


now filtering everything what is below 6s

In [ ]:
df_val[df_val.audio_duration<6]

,client_id,path,sentence,age,gender,accent,locale,validation_status,audio_duration
1,b2c5757d90eeeb699dd436a0bde9c0edd1fcce670f7c48...,common_voice_fi_24001104.mp3,Äänestin mietinnön puolesta!,0,,,fi,valid,3
2,b2c5757d90eeeb699dd436a0bde9c0edd1fcce670f7c48...,common_voice_fi_24001122.mp3,Vain tuntia aikaisemmin olimme mieheni kanssa ...,0,,,fi,valid,5
3,0525d5549e0afaeaf3676f08a0fec54679beea533ad9eb...,common_voice_fi_23997016.mp3,Ensimmäiselle miehelle sai kolme lasta.,0,,,fi,valid,3
5,0525d5549e0afaeaf3676f08a0fec54679beea533ad9eb...,common_voice_fi_23997034.mp3,Tähdenlentojenko varalta minä sen olisin tänne...,0,,,fi,valid,4
6,0525d5549e0afaeaf3676f08a0fec54679beea533ad9eb...,common_voice_fi_23997037.mp3,Siitä se tulee.,0,,,fi,valid,2
...,...,...,...,...,...,...,...,...,...
1300,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,common_voice_fi_23945124.mp3,Kummatkin omissa eväissään.,thirties,male,,fi,valid,3
1301,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,common_voice_fi_23945149.mp3,Paronittaren tapasin hänen sisimmästä huoneest...,thirties,male,,fi,valid,4
1302,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,common_voice_fi_23945153.mp3,Työvuorossa oleva Cassandra avasi oven ja astu...,thirties,male,,fi,valid,5
1303,4eeeb22a3bbb52e5215593a09a845f0f8c496e0a7c498c...,common_voice_fi_23945154.mp3,Mutta kun riihessä liikkui.,thirties,male,,fi,valid,2


In [ ]:
df_val = df_val[df_val.audio_duration<6]
df_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1027 entries, 1 to 1304
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   client_id          1027 non-null   object
 1   path               1027 non-null   object
 2   sentence           1027 non-null   object
 3   age                1027 non-null   object
 4   gender             1027 non-null   object
 5   accent             1027 non-null   object
 6   locale             1027 non-null   object
 7   validation_status  1027 non-null   object
 8   audio_duration     1027 non-null   int64 
dtypes: int64(1), object(8)
memory usage: 80.2+ KB


In [ ]:
df_inval = df_inval[df_inval.audio_duration<6]
df_inval.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 58
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   client_id          40 non-null     object
 1   path               40 non-null     object
 2   sentence           40 non-null     object
 3   age                40 non-null     object
 4   gender             40 non-null     object
 5   accent             40 non-null     object
 6   locale             40 non-null     object
 7   validation_status  40 non-null     object
 8   audio_duration     40 non-null     int64 
dtypes: int64(1), object(8)
memory usage: 3.1+ KB


As both dfs have same columns lets use simple append and create new df to analysys

In [ ]:
df= df_val.append(df_inval)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1067 entries, 1 to 58
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   client_id          1067 non-null   object
 1   path               1067 non-null   object
 2   sentence           1067 non-null   object
 3   age                1067 non-null   object
 4   gender             1067 non-null   object
 5   accent             1067 non-null   object
 6   locale             1067 non-null   object
 7   validation_status  1067 non-null   object
 8   audio_duration     1067 non-null   int64 
dtypes: int64(1), object(8)
memory usage: 83.4+ KB


lets save df for later use

In [ ]:
df.to_hdf('/content/drive/MyDrive/pipeline/finnish_temp.h5', key='df', mode='w') 

# Start from semiprocessed df(after  concatenation)

In [ ]:
df=pd.read_hdf('/content/drive/MyDrive/pipeline/finnish_temp.h5', 'df')

In [ ]:
df.sample(10)

,client_id,path,sentence,age,gender,accent,locale,validation_status,audio_duration
312,fb0d052dcb96d413197d7eb19d6b6c7d63d64f2f4d8d82...,common_voice_fi_22996752.mp3,Aurinkoinen päivä järvellä,0,,,fi,valid,3
615,03350aa4d6bc224e6f519b1ae7b225b66707194de28978...,common_voice_fi_23803406.mp3,"Työnnettyäni nenäni ovesta ulos, sain vahvistu...",fourties,male,,fi,valid,5
699,7839f10ac19c4c5448e198442f4cd7fbfacea0cc2f9d63...,common_voice_fi_23068349.mp3,Mitä siis tekisimme?,twenties,male,,fi,valid,2
320,fb0d052dcb96d413197d7eb19d6b6c7d63d64f2f4d8d82...,common_voice_fi_22996810.mp3,Äänestimme siksi tyhjää tästä mietinnöstä toim...,0,,,fi,valid,5
726,7839f10ac19c4c5448e198442f4cd7fbfacea0cc2f9d63...,common_voice_fi_23178060.mp3,Erämaa kuistin ympärillä tumma.,twenties,male,,fi,valid,2
606,03350aa4d6bc224e6f519b1ae7b225b66707194de28978...,common_voice_fi_23803397.mp3,Hiukan hämillään painoi hän nenäliinaa kasvoil...,fourties,male,,fi,valid,4
16,fb0d052dcb96d413197d7eb19d6b6c7d63d64f2f4d8d82...,common_voice_fi_22996450.mp3,Äänestäjiltä viedään oikeus sanoa viimeinen sana.,0,,,fi,invalid,3
634,03350aa4d6bc224e6f519b1ae7b225b66707194de28978...,common_voice_fi_23803428.mp3,Juttele asiasta enosi kanssa.,fourties,male,,fi,valid,3
385,f6c374d6e4e004c91278cc96d760776b435fb9983e35a7...,common_voice_fi_23102576.mp3,Aurinko oli silloin jo mennyt mailleen.,thirties,male,,fi,valid,3
404,f6c374d6e4e004c91278cc96d760776b435fb9983e35a7...,common_voice_fi_23761546.mp3,"""Mä luulin, et oisit ollu innostuneempi"", Dieg...",thirties,male,,fi,valid,5


Sum of all seconds grouped by age and geneder

In [ ]:
df.groupby([ 'gender', 'age']).sum()

audio_duration
gender age                     
       0                    828
female twenties               8
male   fifties               10
       fourties             634
       thirties            1621
       twenties             615

In [ ]:
df.groupby([ 'gender', 'age']).agg(np.size) #here just numbers of 

client_id  path  ...  validation_status  audio_duration
gender age                        ...                                   
       0               225   225  ...                225             225
female twenties          2     2  ...                  2               2
male   fifties           3     3  ...                  3               3
       fourties        175   175  ...                175             175
       thirties        474   474  ...                474             474
       twenties        188   188  ...                188             188

[6 rows x 7 columns]

In [ ]:
! rm /content/drive/MyDrive/pipeline/temp/*

Posibility of direct(https://gist.github.com/Edresson/752ecaba9b2f10a2b80a2af89fd20d99) link loading  for Mozilla Voce packages was checked but it si no longer available more information   because  of issues like below

https://github.com/common-voice/common-voice/issues/771



Ran into a similar problem and the current accepted solution was too slow for me. My table had 500k+ rows and i needed to update 100k+ rows. After lengthy research and trial and error i arrived at an efficient and correct solution.

The idea is to use psycopg as your writer and to use a temp table. df is your pandas dataframe that contains values you want to set.


```
import psycopg2

conn = psycopg2.connect("dbname='db' user='user' host='localhost' password='test'")
cur = conn.cursor()

rows = zip(df.id, df.z)
cur.execute("""CREATE TEMP TABLE codelist(id INTEGER, z INTEGER) ON COMMIT DROP""")
cur.executemany("""INSERT INTO codelist (id, z) VALUES(%s, %s)""", rows)

cur.execute("""
    UPDATE table_name
    SET z = codelist.z
    FROM codelist
    WHERE codelist.id = vehicle.id;
    """)

cur.rowcount
conn.commit()
cur.close()
conn.close()

```

from http://stackoverflow.com/questions/55052395/ddg#56864146





## Lets see for some dockerised postgress sql

1.
https://testdriven.io/blog/fastapi-docker-traefik/
simple excelent description how to prepare dockerised environment 


whole procedure is nicelly described by Jarek Szczegielniak and presented in article (https://www.codeproject.com/Articles/5302894/Exposing-Dockerized-AI-Models-via-RESTful-API)



https://docs.docker.com/samples/postgresql_service/ maybe this
or set up FastAPI with Postgres, Uvicorn, and Docker.

There couple of things one need to take care when installing Docker on Ubuntu machines( details under this address): (https://www.digitalocean.com/community/tutorials/how-to-install-and-use-docker-on-ubuntu-18-04).

To be able to use dockerised environments for non root users in Ubuntu threre is need to do couple of ajustments. Very extensive reasearch on the subject and helpful tips are on the (https://stackoverflow.com/questions/48957195/how-to-fix-docker-got-permission-denied-issue#comment101604579_48957722) especially part presenting solution of Olshansk user( a least for me)

one more webpage with excelent suggestion to get over problem with executing docekr compose https://gbe0.com/posts/linux/desktop/gdbus-errororg-freedesktop-dbus-error-serviceunknown-the-name-org-freedesktop-secrets-was-not-provided-by-any-service-files/

In [ ]:
df.to

In [ ]:
help(df.to  )